In [ ]:
# Elasticsearchインスタンスの生成

from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
# リスト 3.3.x 日本語向けアナライザ設定とインデックス作成（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・Elasticsearch に日本語検索用の analysis 設定を与える。
#   ・検索時は同義語展開（すし/スシ/鮨/寿司）を適用し、索引時は素直な語彙を保持する設計。
#   ・kuromoji + ICU +（任意のユーザー辞書）で日本語の表記ゆれ/活用/長音などを吸収する。
#
# 設計指針（理論）:
#   ・アナライザは「char_filter → tokenizer → filter」の順で適用される。
#   ・index 時（analyzer="jpn-index"）: 文書をトークン化・正規化して**格納語彙**を作る（通常は同義語なし）。
#   ・search 時（search_analyzer="jpn-search"）: クエリを同様に処理し、**検索側のみ**同義語で想起を拡張する。
#     → 観測性/保守性のため、同義語は検索側に寄せるのが定石（索引側に混ぜない）。
#   ・synonym の "A, B, C" 形式は相互等価展開（expand=true 相当）。片方向正規化は "A, B => C" や synonyms_graph を検討。
#   ・"user_dictionary" は **ESノード上** のパスである点に注意（ローカルPCのパスではない）。
# -----------------------------------------------------------------------------------

# インデックス作成用JSONの定義
create_index = {
    "settings": {
        "analysis": {
            # ---------------------------
            # Token Filter（語彙変換系）の定義
            # ---------------------------
            "filter": {
                "synonyms_filter": {  # 同義語フィルタ（検索側でのみ使用する想定）
                    "type": "synonym",
                    "synonyms": [  # 等価展開: どれで検索しても他の表記を含めて照合する
                        "すし,スシ,鮨,寿司"
                        # 片方向に正規化したい場合の例:
                        # "スシ, すし, 鮨 => 寿司"
                    ],
                }
                # 過正規化（ブランド名等）の保護が必要なら keyword_marker を stemmer 前段に追加する案もある
                # "ja_protected": {
                #   "type": "keyword_marker",
                #   "protected_words": ["メーラー","プレイヤー"]
                # }
            },
            # ---------------------------
            # Tokenizer（形態素分割）の定義
            # ---------------------------
            "tokenizer": {
                "kuromoji_w_dic": {  # ユーザー辞書付き kuromoji トークナイザ
                    "type": "kuromoji_tokenizer",  # ← 綴りは "kuromoji"（コメントの誤記に注意）
                    "user_dictionary": "my_jisho.dic",  # ESノード側の配置パス（*.dic = ビルド済み辞書）
                }
            },
            # ---------------------------
            # Analyzer（前処理パイプライン）の定義
            # ---------------------------
            "analyzer": {
                # 検索用（想起重視: 同義語を含める）
                "jpn-search": {
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer",  # 全/半角・互換文字の正規化（NFKC 等）
                        "kuromoji_iteration_mark",  # 々/ゝ/ヽ など反復記号の展開
                    ],
                    "tokenizer": "kuromoji_w_dic",  # ユーザー辞書込みの形態素解析
                    "filter": [
                        "synonyms_filter",  # ← 同義語展開（検索側のみ）
                        "kuromoji_baseform",  # 活用の原形化（食べた→食べる 等）
                        "kuromoji_part_of_speech",  # 不要品詞除去（助詞・助動詞・記号 等）
                        "ja_stop",  # 日本語ストップワード除去
                        "kuromoji_number",  # 数表現の正規化（漢数字→算用数字 等）
                        "kuromoji_stemmer",  # カタカナ長音の正規化（コンピューター→コンピュータ）
                    ],
                },
                # 索引用（精度/観測性重視: 同義語は含めない）
                "jpn-index": {
                    "type": "custom",
                    "char_filter": ["icu_normalizer", "kuromoji_iteration_mark"],
                    "tokenizer": "kuromoji_w_dic",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_part_of_speech",
                        "ja_stop",
                        "kuromoji_number",
                        "kuromoji_stemmer",
                        # ※ 索引側は素直に保持。検索側で synonyms を適用する方針。
                    ],
                },
            },
        }
    }
}

# 日本語用インデックス名の定義
jp_index = "jp_index"

# 既存インデックスの削除（学習/検証用途）
# 本番では必ずエイリアス切替 + スナップショット等の安全策を取ること。
if es.indices.exists(index=jp_index):
    es.indices.delete(index=jp_index)

# インデックスの作成（settings のみ）
# この後、マッピングで対象フィールドへ
#   "analyzer": "jpn-index", "search_analyzer": "jpn-search"
# を割り当てないと効果が出ない点に注意（content 等の text フィールドに明示する）。
es.indices.create(index=jp_index, body=create_index)

# 【動作確認のヒント（必要時にコメント解除）】
# 1) クエリ側トークン（同義語展開を含む）を確認
# print(es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"寿司"}))
# 2) マッピングの例（参考）
# mapping = {"properties":{"content":{"type":"text","analyzer":"jpn-index","search_analyzer":"jpn-search"}}}
# es.indices.put_mapping(index=jp_index, body=mapping)

In [ ]:
# 分析結果表示用関数（詳細コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・Elasticsearch の _analyze API を用いて、指定インデックス（jp_index）に定義済みの
#     検索用アナライザ "jpn-search" で日本語テキストを解析し、生成トークン（語彙）だけを配列で返す。
#
# 前提:
#   ・変数 es: Elasticsearch クライアントが初期化済み（例: es = Elasticsearch(...)）。
#   ・変数 jp_index: 解析対象インデックス名の文字列（例: 'jp_index'）。
#   ・インデックス jp_index 側の settings.analysis に "jpn-search" アナライザが定義されていること。
#     （char_filter → tokenizer(kuromoji) → filter(synonyms/baseform/POS/stop/number/stemmer) のような構成）
#
# 出力:
#   ・list[str]: 各トークンの "token" 文字列のみを抽出した配列。
#
# 備考（理論）:
#   ・_analyze は「char_filter → tokenizer → filter」の順に適用された“検索前処理”の結果を返す。
#   ・返却 JSON の tokens は、各要素に token / position / start_offset / end_offset / type などを含む。
#     本関数は学習用途のため token 文字列のみを抽出する最小実装としている。
#   ・検索時は通常、インデックス側は "jpn-index"（同義語なし）、クエリ側は "jpn-search"（同義語あり）
#     と分離する設計が観測性・保守性の観点で推奨される。
# -----------------------------------------------------------------------------


def analyse_jp_text(text):
    """
    与えられた文字列を "jpn-search" アナライザで解析し、得られた語彙（token）配列を返す。
    例外は上位へ伝播（RequestError/ConnectionError など）する。
    """
    # _analyze API に渡すリクエストボディを構築
    # - analyzer: 検索用アナライザ名を明示（"jpn-search"）
    # - text    : 解析対象の生テキスト
    body = {"analyzer": "jpn-search", "text": text}

    # 解析実行:
    # - index を指定することで、そのインデックスに紐づく analysis（ユーザー辞書・同義語など）が反映される
    # - 代表的な失敗例:
    #   * 400 Bad Request: "jpn-search" 未定義 / analysis 設定の不整合 / ユーザー辞書読み込み失敗
    #   * 接続例外       : ES ノード未起動・接続設定不備
    ret = es.indices.analyze(index=jp_index, body=body)

    # レスポンス構造の例:
    # {
    #   "tokens": [
    #     {"token": "寿司", "start_offset": 0, "end_offset": 2, "type": "...", "position": 0},
    #     ...
    #   ]
    # }
    tokens = ret["tokens"]

    # 各トークン辞書から "token" フィールド（語彙文字列）のみを抽出
    tokens2 = [token["token"] for token in tokens]

    # 学習用途として単純な配列で返す（詳細が必要なら position/offset を返す拡張版を別関数で用意すると良い）
    return tokens2


# 関数のテスト:
# ・設定に依存するが、助詞/記号は POS/ja_stop で落ち、内容語が中心に残ることが多い。
# ・同義語（例: すし/スシ/鮨/寿司）を "jpn-search" に定義している場合は、クエリ側の表記ゆれ吸収を確認できる。
print(analyse_jp_text("関数のテスト"))

# （デバッグ補助: クエリ側の実トークンを詳細に確認したい場合の例。必要時のみコメント解除）
# toks = es.indices.analyze(index=jp_index, body={"analyzer": "jpn-search", "text": "関数のテスト"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))

In [ ]:
#  リスト3.3.20 辞書登録後の「新幹線はやぶさ」と「はやぶさ」の分析結果

print(analyse_jp_text("新幹線はやぶさ"))
print(analyse_jp_text("はやぶさ"))

In [ ]:
# リスト 3.3.12（改）マッピング定義（詳細コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・インデックス jp_index に対し、全文検索対象フィールド "content" のマッピングを設定する。
#   ・投入時は analyzer="jpn-index"（索引用チェーン）、検索時は search_analyzer="jpn-search"
#     （検索用チェーン: 同義語などで想起拡張）を適用して、表記ゆれに強く観測性の高い設計にする。
#
# 理論ポイント:
#   ・Elasticsearch v6+ では全文検索用は "text"、完全一致や集計用は "keyword" を使う。
#   ・"analyzer" は **索引時**（ドキュメント投入時）に適用、"search_analyzer" は **検索時**（クエリ解析）
#     に適用されるため、両者を分離すると「索引は素直に、検索で拡張」という運用が可能になる。
#   ・既存フィールドの analyzer を **後から変更することはできない**。変更が必要なら
#     新インデックス作成 → _reindex → エイリアス切替、が基本手順。
#   ・title や name など他フィールドに検索/集計要件があるなら、それぞれの型・アナライザも明示定義する。
#   ・完全一致/ソート/集計のために "content.raw" のような keyword のマルチフィールドを併設するのが定石。
# -----------------------------------------------------------------------------

mapping = {
    "properties": {
        "content": {
            # 全文検索対象として "text" 型を使用（v5 までの "string" は廃止済み）
            "type": "text",
            # ドキュメント投入（index）時のアナライザ
            # - "jpn-index": kuromoji + 各種フィルタ（通常は同義語なし）
            # - 索引側を“素直”に保つことで、語彙の観測性と再構築容易性を確保
            "analyzer": "jpn-index",
            # クエリ解析（search）時のアナライザ
            # - "jpn-search": 同義語展開（例: すし/スシ/鮨/寿司）などで想起を拡張し、表記ゆれを吸収
            "search_analyzer": "jpn-search",
            # （必要に応じて追加する例：完全一致や集計用の keyword サブフィールド）
            # "fields": {
            #   "raw": {
            #     "type": "keyword",
            #     "ignore_above": 256
            #     # 必要に応じて normalizer を定義（lowercase など）
            #   }
            # }
        }
    }
}

# マッピングの適用
# - 既存インデックスに対しては「新規フィールド追加」等は可能だが、
#   既存フィールドの analyzer 変更は不可。必要なら新インデックス移行を行うこと。
es.indices.put_mapping(index=jp_index, body=mapping)

# （確認ヒント：必要時にコメント解除）
# cur = es.indices.get_mapping(index=jp_index)
# from pprint import pprint
# pprint(cur[jp_index]["mappings"]["properties"].get("content"))

In [ ]:
# リスト 3.3.13（再掲・詳細コメント付き）日本語文書の投入
# -----------------------------------------------------------------------------------
# 目的:
#   ・前段で作成済みの日本語向けインデックス jp_index に、サンプル文書を投入（index）する。
#
# 前提/設計:
#   ・"content" フィールドはマッピングで
#       analyzer="jpn-index"（索引時） / search_analyzer="jpn-search"（検索時）
#     を設定済み（= 投入時は文字正規化→kuromoji→原形化/stop/長音などが適用され、語彙が安定化）。
#   ・検索時は "jpn-search" 側で同義語（例: すし/スシ/鮨/寿司）等を適用し、想起を拡張する方針。
#
# 実務上の注意:
#   1) 動的マッピング:
#      - 本スニペットでは title, name の明示マッピングは未指定 → 動的マッピングに従う。
#        これらを検索/集計に使うなら text/keyword 等を明示定義するのが望ましい。
#   2) 上書き挙動:
#      - es.index(index=..., id=...) は同一IDが存在すれば上書き（バージョン増分）となる。
#        既存IDがあれば失敗させたいときは op_type="create" を使う。
#   3) 即時検索の可視化:
#      - 直後に検索するなら refresh="wait_for" を付けるか、事後に es.indices.refresh(index=...) を呼ぶ。
#   4) パフォーマンス:
#      - 大量投入では elasticsearch.helpers.bulk の利用を推奨。
#   5) 互換性:
#      - elasticsearch-py 8.x では body より document 引数が推奨（後方互換のため body でも動作）。
# -----------------------------------------------------------------------------------

bodys = [
    {  # ドキュメントID: 0
        "title": "山田太郎の紹介",
        "name": {"last": "山田", "first": "太郎"},
        # 例: 「スシ」は icu_normalizer の互換正規化や kuromoji の形態素解析、
        #     さらに stemmer/stop 等により、索引側の語彙として安定化して格納される。
        "content": "スシが好物です。犬も好きです。",
    },
    {  # ドキュメントID: 1
        "title": "田中次郎の紹介",
        "name": {"last": "田中", "first": "次郎"},
        # 例: 「だいすき/大好き」の表記ゆれは正規化＋原形化である程度吸収。
        #     固有表現（新幹線名など）はユーザー辞書の整備で分割の安定性が向上する。
        "content": "そばがだいすきです。ねこも大好きです。",
    },
    {  # ドキュメントID: 2
        "title": "渡辺三郎の紹介",
        "name": {"last": "渡辺", "first": "三郎"},
        # 例: 「はやぶさ」は固有名詞として 1 トークン化されやすいが辞書依存。
        "content": "天ぷらが好きです。新幹線はやぶさのファンです。",
    },
]

# 文書投入ループ
# - enumerate により _id=0,1,2 を割り当て。
# - 直後に検索する検証を行うなら refresh="wait_for" を付けると可視化の遅延を避けられる。
for i, body in enumerate(bodys):
    es.index(index=jp_index, id=i, body=body)
    # 例（8.x 推奨の引数スタイル & 衝突回避・即時可視化の例）:
    # es.index(index=jp_index, id=i, document=body, op_type="create", refresh="wait_for")

# 直後に検索する場合の明示リフレッシュ（上の refresh を使わない場合の代替）
# es.indices.refresh(index=jp_index)

# （参考）大量投入の雛形:
# from elasticsearch import helpers
# actions = ({"_index": jp_index, "_id": i, "_source": doc} for i, doc in enumerate(bodys))
# helpers.bulk(es, actions, refresh="wait_for")

In [ ]:
# リスト 3.3.21 辞書登録後にキーワード「はやぶさ」で検索（詳細コメント付き）
# -----------------------------------------------------------------------------------
# 目的:
#   ・ユーザー辞書（my_jisho.dic）を組み込んだ kuromoji トークナイザ設定（"kuromoji_w_dic"）を
#     インデックス jp_index に反映させた後（＝「辞書登録後」）に、
#     検索用アナライザ "jpn-search" でクエリ語「はやぶさ」を全文検索（match）し、ヒット結果を確認する。
#
# 重要ポイント（理論）:
#   1) アナライザの適用:
#      - 検索時は search_analyzer="jpn-search" がクエリ文字列に適用される。
#        処理順: char_filter（icu_normalizer 等）→ tokenizer（kuromoji_w_dic）→ filter
#        （synonyms/baseform/POS/stop/number/stemmer）
#   2) ユーザー辞書の効果:
#      - 「はやぶさ」単体は既定辞書でも 1 トークンになりやすいが、
#        連接語（例: 「新幹線はやぶさ」）の分割安定化・品詞付与の改善にユーザー辞書が効く。
#      - **辞書や analysis 設定の変更は、既に格納済みのトークンには後追いで反映されない。**
#        → 変更反映には「新インデックス作成 → _reindex → エイリアス切替」等が必要。
#   3) 検索意味論:
#      - `match` は token-level の全文検索。BM25（TF・IDF・文書長補正）でスコアリング。
#      - 語順・近接を重視するなら `match_phrase`、複数フィールド重み付けなら `multi_match` を検討。
#   4) 可視化の安定化:
#      - 直前に index した文書を即検索で確認する場合、index 側で `refresh="wait_for"` を使うか、
#        検索前に `es.indices.refresh(index=jp_index)` を行う。
#   5) 同義語について:
#      - 本系の同義語定義は「寿司/すし/スシ/鮨」であり、「はやぶさ」には同義語展開は掛からない想定（no-op）。
#        固有名詞の別表記を束ねたい場合は synonyms を別途設計する。
# -----------------------------------------------------------------------------------

# 検索条件の設定
query = {
    "query": {
        "match": {
            "content": "はやぶさ"  # ← 検索側で jpn-search が適用され、kuromoji_w_dic に基づきトークナイズされる
        }
    }
}

# （参考: 8.x 推奨の引数スタイルと厳密件数の明示。必要に応じて置き換え）
# res = es.search(
#     index=jp_index,
#     query={"match": {"content": "はやぶさ"}},
#     size=20,
#     track_total_hits=True,
#     _source=["title","name.last","content"]
# )

# 検索実行（後方互換のため body=... でも可）
res = es.search(index=jp_index, body=query)

# 結果表示
import json

print(json.dumps(res, indent=2, ensure_ascii=False))

# -----------------------------------------------------------------------------------
# デバッグ補助（必要時のみコメント解除）:
# 1) クエリ側トークンの確認（辞書・stop・原形化の影響を把握）
# toks = es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"はやぶさ"})["tokens"]
# for t in toks:
#     print(t["token"], t.get("position"), t.get("start_offset"), t.get("end_offset"))
#
# 2) 直前投入文書の可視化（refresh）
# es.indices.refresh(index=jp_index)
#
# 3) フレーズ一致（語順・近接を重視：連接語を狙い撃ち）
# res = es.search(index=jp_index, query={"match_phrase": {"content": "新幹線 はやぶさ"}}, size=20, track_total_hits=True)
# print(json.dumps(res, indent=2, ensure_ascii=False))
#
# 4) 再インデックスの注意:
# - ユーザー辞書の更新や analysis の変更は既存トークンに遡及しないため、
#   新インデックス（更新済み settings + mappings）を作り、_reindex で移行するのが基本。
# -----------------------------------------------------------------------------------